In [1]:
import pathlib
import sys
import shutil
import tempfile
import os
import subprocess
import csv
import biom
import numpy as np
import pandas as pd
from qiime2 import Artifact, Metadata
from qiime2.plugins import cutadapt 
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.feature_table.visualizers import summarize as summarize_table

In [2]:
metadata = Metadata.load('../IndexJumping/Sample_Map.txt')
demux = Artifact.load('../IndexJumping/demultiplexed_seqs.qza')

In [3]:
#Trimming ANML primers off the demultiplexed reads
ANML_primers = ["^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC"]

trimmed_demux = cutadapt.methods.trim_single(demux, 
                                             cores=16, 
                                             adapter = ANML_primers, 
                                             indels = True,
                                             minimum_length = 170, 
                                             discard_untrimmed = True)


Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F01R11_CTAAGGTAACGATGATTCGAGGA_L001_R1_001.fastq.gz --adapter ^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC --discard-untrimmed /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-zgkuymgn/db4f0261-44f7-473c-ba38-c72c16af0e2a/data/F01R11_CTAAGGTAACGATGATTCGAGGA_L001_R1_001.fastq.gz

This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F01R11_CTAAGGTAACGATGATTCGAGGA_L

No reads processed!

Command: cutadapt --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F01R17_CTAAGGTAACGATGACGAATAGA_L001_R1_001.fastq.gz --adapter ^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC --discard-untrimmed /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-zgkuymgn/db4f0261-44f7-473c-ba38-c72c16af0e2a/data/F01R17_CTAAGGTAACGATGACGAATAGA_L001_R1_001.fastq.gz

This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F01R17_CTAAGGTAACGATGACGAATAGA_L001_R1_001.fastq.gz --adapter ^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC --discard-untrimmed /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-zgkuymgn/db4f0261-44f7-473c-ba38-c72c1

Finished in 0.42 s (68 µs/read; 0.89 M reads/minute).

=== Summary ===

Total reads processed:                   6,250
Reads with adapters:                     6,243 (99.9%)

== Read fate breakdown ==
Reads that were too short:                  30 (0.5%)
Reads discarded as untrimmed:                7 (0.1%)
Reads written (passing filters):         6,213 (99.4%)

Total basepairs processed:     1,450,020 bp
Total written (filtered):      1,126,449 bp (77.7%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 6243 times; 3' trimmed: 6231 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
23	1	0.0	2	0 0 1
24	66	0.0	2	0 55 9 2
25	6148	0.0	2	5625 477 45 1
26	28	0.0	2	0 18 10



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts
23	1	0.0	2	1
24	1

Finished in 0.36 s (72930 µs/read; 0.00 M reads/minute).

=== Summary ===

Total reads processed:                       5
Reads with adapters:                         5 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   0 (0.0%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):             5 (100.0%)

Total basepairs processed:         1,163 bp
Total written (filtered):            908 bp (78.1%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 5 times; 3' trimmed: 5 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
25	5	0.0	2	5



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts
26	5	0.0	2	5

Command: cutadapt --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum

Finished in 0.36 s (360534 µs/read; 0.00 M reads/minute).

=== Summary ===

Total reads processed:                       1
Reads with adapters:                         1 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   0 (0.0%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):             1 (100.0%)

Total basepairs processed:           232 bp
Total written (filtered):            181 bp (78.0%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 1 times; 3' trimmed: 1 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
25	1	0.0	2	1



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts
26	1	0.0	2	1

Command: cutadapt --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimu

...

Finished in 0.42 s (67 µs/read; 0.90 M reads/minute).

=== Summary ===

Total reads processed:                   6,250
Reads with adapters:                     6,241 (99.9%)

== Read fate breakdown ==
Reads that were too short:                  63 (1.0%)
Reads discarded as untrimmed:                9 (0.1%)
Reads written (passing filters):         6,178 (98.8%)

Total basepairs processed:     1,443,949 bp
Total written (filtered):      1,122,098 bp (77.7%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 6241 times; 3' trimmed: 6215 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
23	5	0.0	2	0 0 5
24	52	0.0	2	0 45 6 1
25	6161	0.0	2	5551 523 86 1
26	22	0.0	2	0 12 9 1
27	1	0.0	2	0 0 1



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts

Finished in 0.41 s (66 µs/read; 0.91 M reads/minute).

=== Summary ===

Total reads processed:                   6,250
Reads with adapters:                     6,246 (99.9%)

== Read fate breakdown ==
Reads that were too short:                  43 (0.7%)
Reads discarded as untrimmed:                4 (0.1%)
Reads written (passing filters):         6,203 (99.2%)

Total basepairs processed:     1,450,794 bp
Total written (filtered):      1,125,733 bp (77.6%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 6246 times; 3' trimmed: 6230 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
23	1	0.0	2	0 0 1
24	58	0.0	2	0 50 7 1
25	6168	0.0	2	5519 592 57
26	16	0.0	2	0 10 6
27	3	0.0	2	0 0 3



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts
23	

No reads processed!

Command: cutadapt --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F10R12_CTGACCGAACGATGAACCACCTA_L001_R1_001.fastq.gz --adapter ^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC --discard-untrimmed /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-zgkuymgn/db4f0261-44f7-473c-ba38-c72c16af0e2a/data/F10R12_CTGACCGAACGATGAACCACCTA_L001_R1_001.fastq.gz

This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 16 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 170 -o /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-flgo81t7/F10R12_CTGACCGAACGATGAACCACCTA_L001_R1_001.fastq.gz --adapter ^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC --discard-untrimmed /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-zgkuymgn/db4f0261-44f7-473c-ba38-c72c1

Finished in 0.44 s (71 µs/read; 0.85 M reads/minute).

=== Summary ===

Total reads processed:                   6,250
Reads with adapters:                     6,247 (100.0%)

== Read fate breakdown ==
Reads that were too short:               3,626 (58.0%)
Reads discarded as untrimmed:                3 (0.0%)
Reads written (passing filters):         2,621 (41.9%)

Total basepairs processed:       818,835 bp
Total written (filtered):        480,637 bp (58.7%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 6247 times; 3' trimmed: 5969 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
23	6	0.0	2	0 0 2 2 2
24	91	0.0	2	0 69 13 9
25	6125	0.0	2	4232 1066 822 5
26	19	0.0	2	0 8 6 4 1
27	6	0.0	2	0 0 5 0 1



Overview of removed sequences at 3' end
length	count	expect	max.er

Finished in 0.37 s (123871 µs/read; 0.00 M reads/minute).

=== Summary ===

Total reads processed:                       3
Reads with adapters:                         3 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   0 (0.0%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):             3 (100.0%)

Total basepairs processed:           698 bp
Total written (filtered):            545 bp (78.1%)

=== Adapter 3 ===

Sequence: GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC; Type: linked; Length: 25+26; 5' trimmed: 3 times; 3' trimmed: 3 times

Minimum overlap: 25+3
No. of allowed errors: 2

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-26 bp: 2

Overview of removed sequences at 5' end
length	count	expect	max.err	error counts
25	3	0.0	2	3



Overview of removed sequences at 3' end
length	count	expect	max.err	error counts
26	3	0.0	2	3


In [4]:
summarize(trimmed_demux.trimmed_sequences).visualization

<visualization: Visualization uuid: c61f6e7c-1df1-4b4b-b5e0-cbfcf9420f18>

In [5]:
#DADA2
feature_table, rep_seqs, stats = denoise_single(trimmed_demux.trimmed_sequences,
                                                trunc_len = 0, 
                                                n_threads = 0)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/qiime2-archive-3yyuz436/e64260d4-c583-4db5-bce3-ed858c0daca0/data /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/output.tsv.biom /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/track.tsv /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx 0 0 2.0 2 Inf independent consensus 1.0 0 1000000 NULL 16

R version 4.0.5 (2021-03-31) 


Loading required package: Rcpp


DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering 

The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F01R18_CTAAGGTAACGATGCAATTGCCT_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F01R14_CTAAGGTAACGATGACACTCCAA_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F01R16_CTAAGGTAACGATGTCATCCAGA_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F01R13_CTAAGGTAACGATGTCCGTTAGA_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F02R11_TAAGGAGAACGATGATTCGAGGA_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9kzw1ptc0000gn/T/tmpc_oxa8rx/F01R17_CTAAGGTAACGATGACGAATAGA_L001_R1_001.fastq.gz not written.
The filter removed all reads: /var/folders/hh/1l3l9sld7h5dffby9k

..xx.xxxxxx...................x.........x.........x.........x.........x.........x.........x.........
2) Learning Error Rates
60837974 total bases in 377153 reads from 85 samples will be used for learning the error rates.
3) Denoise samples .....................................................................................
4) Remove chimeras (method = consensus)
5) Report read numbers through the pipeline
6) Write output


In [6]:
feature_table.save('../PerSampleFiltering/feature_table.qza')
summarize_table(feature_table).visualization

<visualization: Visualization uuid: e0d05147-ed27-4bab-98a8-0ce43f87c9fb>

In [6]:
# Extract .biom from feature table file if necessary
def extract_all(file, dest):
    with tempfile.TemporaryDirectory() as temp:
        file.export_data(temp)
        temp_pathlib = pathlib.Path(temp)
        for file in temp_pathlib.iterdir():
            shutil.copy(file, dest)
extract_all(feature_table, 'out')